# Prompt enginnering & Backend

## Import and Ingest Data

In [1]:
from dotenv import load_dotenv
import os
import json
load_dotenv()
H2O_API_KEY = os.getenv("H2O_API_KEY") 
from h2ogpte import H2OGPTE

In [4]:
def init_program(data_dir):
    client = H2OGPTE(
        address='https://h2ogpte.genai.h2o.ai',
        api_key= H2O_API_KEY,
    )

    # Create a new collection
    collection_id = client.create_collection(
        name='devMeetingAssistant',
        description='Test development for meeting assistant (prompt engineering)',
    )
    with open(data_dir, 'rb') as f:
        meeting_data1 = client.upload('Data/fake_sample.txt', f)
    client.ingest_uploads(collection_id, [meeting_data1,])
    chat_session_id = client.create_chat_session(collection_id)
    return client, collection_id

def rag_chat(client, chat_session_id, llm_type, main_prompt, system_prompt):
    with client.connect(chat_session_id) as session:
        answer = session.query(
            message=main_prompt,
            system_prompt=system_prompt,
            rag_config={
            "rag_type": "rag", # https://h2oai.github.io/h2ogpte/getting_started.html#advanced-controls-for-document-q-a
            },
            llm=llm_type,
        )
        return answer

def extract_json_string(full_string): 
    """Extracts json-like string from LLM response. Outputs str."""

    start_index = full_string.find('{')  # Find the index of the first '{'
    end_index = full_string.rfind('}')   # Find the index of the last '}'
    
    if start_index != -1 and end_index != -1:
        return full_string[start_index:end_index + 1]
    else:
        return None
    
def string_like_JSON_to_txt(json_str):
    """Input: json-like string
    output: Email body string"""
    if json_str is None:
        return "Invalid JSON format. Cannot parse the response."

    # Parse the extracted JSON string
    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        return "Error parsing JSON data."

    # Extract information from the parsed data
    agenda = data.get('Agenda', '')
    meeting_summary = data.get('Meeting Summary', '')
    actionables = data.get('Actionables', [])

    # Format the post-meeting email
    email_subject = f"Post-Meeting Summary: {agenda}"
    email_body = f"Dear Team,\n\nHere's a summary of our recent meeting:\n\nAgenda: {agenda}\n\nMeeting Summary:\n{meeting_summary}\n\nActionables:\n"
    for action in actionables:
        email_body += f"- {action['Action']} (Assigned to: {action['Assigned']}, Deadline: {action['Deadline']}, Priority: {action['Priority']})\n"

    email_body += "\nPlease let me know if anything needs clarification or if there are additional action items to add.\n\nBest regards,\n[Your Name]"
    return email_body

In [5]:
def get_summary_response(client, chat_session_id, llm_type):
    system_prompt = """
    You are a Meeting Assistant for a large company that wants to maximize meeting efficiency. 
    You will be given meeting transcript(s) from real project meetings held over zoom.
    Follow all the instructions closely or everyone in the team will be fired.
    If you do well, you will be rewarded with a $1000 tip.
    """
    main_prompt = """
    Execute all the following steps: \
    1. Examine the document and identify all names of the attendees, remember these names for the next steps.
    2. Identify and extract the main purpose or agenda of the meeting
    3. Identify all actionables mentioned during the meeting
    4. Extract all the actionables and action plans, identify all details related to it including the deadlines, who it is assigned to, etc.
    5. Summarize the entire transcript
    6. Present all identified information in the following JSON format for easy parsing:
    {"Agenda": "", "Meeting Summary":"", "Actionables":[{"Action":"", "Deadline":"", "Assigned":""}, ...]}
    DO NOT include anything other than the JSON format output in your response.
    """
    return rag_chat(client, chat_session_id, llm_type, main_prompt, system_prompt)

def get_sentiment_response(client, chat_session_id, llm_type, meeting_summary_response):
    system_prompt = """
    You are a Meeting Assistant for a large company that wants to maximize meeting efficiency. 
    You will be given meeting transcript(s) from real project meetings held over zoom.
    Follow all the instructions closely or everyone in the team will be fired.
    If you do well, you will be rewarded with a $1000 tip.
    """
    sentiment_prompt = f"""
    Execute all the following steps: \
    1. Examine the contexts of this JSON object: {meeting_summary_response}
    2. For each actionable, execute the following steps
    a. identify the sentence in the document where it is mentioned
    b. Interpret the sentence, based on this context, categorize the priority of this task as "High", "Medium" or "Low"
    c. Edit the JSON object, add a column to the dictionary for this action as follows: "Priority": ""
    3. Present all identified information in the following JSON format:
    {{"Agenda": "", "Meeting Summary":""}} for the entire meeting
    {{"Actionables":[{{"Action":"", "Deadline":"", "Assigned":"", "Priority":""}}]}} for each action plan
    """

    return rag_chat(client, chat_session_id, llm_type, sentiment_prompt, system_prompt)



In [53]:
def get_LLM_response(data_dir, llm_type):
    """
    Output: json object
    """
    client, collection_id = init_program(data_dir)
    chat_session_id = client.create_chat_session(collection_id)
    # 1. Summary and action plan
    meeting_summary_response = get_summary_response(client, chat_session_id, llm_type)

    # 2. Sentiment analysis/Priority allocation
    sentiment_reply = get_sentiment_response(client, chat_session_id, llm_type ,meeting_summary_response)

    # Convert to email body
    return string_like_JSON_to_txt(sentiment_reply)

In [6]:
data_dir = "Data/fake_sample.txt"

In [54]:
email_reply_llama70 = get_LLM_response(data_dir,llm_type='h2oai/h2ogpt-4096-llama2-70b-chat')
print(email_reply_llama70)

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.4.7`.
You can enable strict version checking by passing strict_version_check=True.
Invalid JSON format. Cannot parse the response.


In [7]:
def test_response(data_dir, llm_type):
    client, collection_id = init_program(data_dir)
    chat_session_id = client.create_chat_session(collection_id)
    # 1. Summary and action plan
    meeting_summary_response = get_summary_response(client, chat_session_id, llm_type)
    
    sentiment_response = get_sentiment_response(client, chat_session_id, llm_type, extract_json_string(meeting_summary_response.content))
    print(sentiment_response.content)
    return string_like_JSON_to_txt(sentiment_response)

In [8]:
test_response = test_response(data_dir, llm_type='h2oai/h2ogpt-4096-llama2-70b-chat')
print(test_response)

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.4.9`.
You can enable strict version checking by passing strict_version_check=True.
{
"Agenda": "Reviewing progress on the project, discussing roadblocks, and planning next steps",
"Meeting Summary": "The meeting discussed the current status of the project, reviewed the design proposal, and finalized the budget allocation. The team also discussed potential roadblocks and found solutions, and scheduled a follow-up discussion. The meeting concluded with a summary of actionables and their deadlines.",
"Actionables": [
{
"Action": "Complete design proposal",
"Deadline": "Wednesday",
"Assigned": "Ryan_Edward",
"Priority": "High"
},
{
"Action": "Confirm meeting with design specialist",
"Deadline": "Thursday",
"Assigned": "Ben_CH",
"Priority": "Medium"
},
{
"Action": "Resolve integration issue",
"Deadline": "Friday",
"Assigned": "Chan Yi Ru Micole",
"Priority": "Medium"
},
{
"Action": "Share progress document",
"Deadlin

In [60]:
test_response = test_response(data_dir, llm_type='h2oai/h2ogpt-4096-llama2-70b-chat')
print(test_response)

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.4.7`.
You can enable strict version checking by passing strict_version_check=True.
{
"Agenda": "Reviewing progress on the project, discussing roadblocks, and planning next steps",
"Meeting Summary": "The meeting discussed the current status of the project, reviewed the design proposal, and finalized the budget allocation. The team also discussed potential roadblocks and found solutions, and scheduled a follow-up discussion. The meeting concluded with a summary of actionables and their deadlines.",
"Actionables": [
{
"Action": "Complete design proposal",
"Deadline": "Wednesday",
"Assigned": "Ryan_Edward",
"Priority": "High"
},
{
"Action": "Confirm meeting with design specialist",
"Deadline": "Thursday",
"Assigned": "Ben_CH",
"Priority": "Medium"
},
{
"Action": "Resolve integration issue",
"Deadline": "Friday",
"Assigned": "Chan Yi Ru Micole",
"Priority": "Medium"
},
{
"Action": "Share progress document",
"Deadlin

In [ ]:
response= """{
"Agenda": "Reviewing progress on the project, discussing roadblocks, and planning next steps",
"Meeting Summary": "The meeting discussed the current status of the project, including completed tasks and ongoing challenges. The team reviewed the project timeline and identified potential roadblocks, such as ensuring design alignment with industry standards. Action plans were established, including scheduling a meeting with a design specialist and finalizing the budget allocation. The team also discussed the integration issue and testing progress."
"Actionables": [
{
"Action": "Schedule a meeting with a design specialist",
"Deadline": "This week",
"Assigned": "Ben"
},
{
"Action": "Finalize budget allocation",
"Deadline": "End of the week",
"Assigned": "Ben"
},
{
"Action": "Complete design proposal",
"Deadline": "Wednesday",
"Assigned": "Ryan"
},
{
"Action": "Testing and resolve integration issue",
"Deadline": "Friday",
"Assigned": "Micole"
}
]
}

The attendees' names are:

* Eugene_YJ
* Ryan_Edward
* Ben_CH
* Chan Yi Ru Micole"""
json_str = json.loads(extract_json_string(response))


In [36]:
print(meet_response)

Error parsing JSON data.


In [23]:
meet_response = test_response(data_dir, llm_type='h2oai/h2ogpt-4096-llama2-70b-chat')
print(meet_response.content)

{
"Agenda": "Reviewing progress on the project, discussing roadblocks, and planning next steps",
"Meeting Summary": "The meeting discussed the current status of the project, including completed tasks and ongoing challenges. The team reviewed the project timeline and identified potential roadblocks, such as ensuring design alignment with industry standards. Action plans were established, including scheduling a meeting with a design specialist and finalizing the budget allocation. The team also discussed the integration issue and testing progress."
"Actionables": [
{
"Action": "Schedule a meeting with a design specialist",
"Deadline": "This week",
"Assigned": "Ben"
},
{
"Action": "Finalize budget allocation",
"Deadline": "End of the week",
"Assigned": "Ben"
},
{
"Action": "Complete design proposal",
"Deadline": "Wednesday",
"Assigned": "Ryan"
},
{
"Action": "Testing and resolve integration issue",
"Deadline": "Friday",
"Assigned": "Micole"
}
]
}

The attendees were:
Eugene_YJ
Ryan_Edward

In [ ]:
"""{'base_model': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'prompt_type': 'mixtral', 'prompt_dict': {'promptA': '<<SYS>>\nYou are an AI that follows instructions extremely well and as helpful as possible.\n<</SYS>>\n\n', 'promptB': '<<SYS>>\nYou are an AI that follows instructions extremely well and as helpful as possible.\n<</SYS>>\n\n', 'PreInstruct': '<s> [INST] ', 'PreInput': None, 'PreResponse': '[/INST]', 'terminate_response': ['[INST]', '</s>'], 'chat_sep': ' ', 'chat_turn_sep': '</s> ', 'humanstr': '[INST]', 'botstr': '[/INST]', 'generates_leading_space': False, 'system_prompt': 'You are an AI that follows instructions extremely well and as helpful as possible.', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 32718, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'h2oai/h2ogpt-4096-llama2-70b-chat', 'prompt_type': 'llama2', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': "<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\n", 'PreInput': None, 'PreResponse': '[/INST]', 'terminate_response': ['[INST]', '</s>'], 'chat_sep': ' ', 'chat_turn_sep': ' </s>', 'humanstr': '[INST]', 'botstr': '[/INST]', 'generates_leading_space': False, 'system_prompt': "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.", 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 4046, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'h2oai/h2ogpt-4096-llama2-13b-chat', 'prompt_type': 'llama2', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': "<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\n", 'PreInput': None, 'PreResponse': '[/INST]', 'terminate_response': ['[INST]', '</s>'], 'chat_sep': ' ', 'chat_turn_sep': ' </s>', 'humanstr': '[INST]', 'botstr': '[/INST]', 'generates_leading_space': False, 'system_prompt': "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.", 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 4046, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'h2oai/h2ogpt-32k-codellama-34b-instruct', 'prompt_type': 'llama2', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': "<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\n", 'PreInput': None, 'PreResponse': '[/INST]', 'terminate_response': ['[INST]', '</s>'], 'chat_sep': ' ', 'chat_turn_sep': ' </s>', 'humanstr': '[INST]', 'botstr': '[/INST]', 'generates_leading_space': False, 'system_prompt': "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.", 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 32718, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'HuggingFaceH4/zephyr-7b-beta', 'prompt_type': 'zephyr', 'prompt_dict': {'promptA': '<|system|>\nYou are an AI that follows instructions extremely well and as helpful as possible.</s>\n', 'promptB': '<|system|>\nYou are an AI that follows instructions extremely well and as helpful as possible.</s>\n', 'PreInstruct': '<|user|>\n', 'PreInput': None, 'PreResponse': '</s>\n<|assistant|>\n', 'terminate_response': ['<|assistant|>', '</s>'], 'chat_sep': '', 'chat_turn_sep': '</s>\n', 'humanstr': '<|user|>', 'botstr': '<|assistant|>', 'generates_leading_space': False, 'system_prompt': 'You are an AI that follows instructions extremely well and as helpful as possible.', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 32718, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'NousResearch/Nous-Capybara-34B', 'prompt_type': 'vicuna11', 'prompt_dict': {'promptA': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. ", 'promptB': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. ", 'PreInstruct': 'USER: ', 'PreInput': None, 'PreResponse': 'ASSISTANT:', 'terminate_response': ['ASSISTANT:', '</s>'], 'chat_sep': ' ', 'chat_turn_sep': '</s>', 'humanstr': 'USER: ', 'botstr': 'ASSISTANT:', 'generates_leading_space': False, 'system_prompt': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.", 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 199950, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'claude-2.1', 'prompt_type': 'anthropic', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': '', 'PreInput': None, 'PreResponse': '', 'terminate_response': [], 'chat_sep': '\n', 'chat_turn_sep': '\n', 'humanstr': None, 'botstr': None, 'generates_leading_space': False, 'system_prompt': '', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 199950, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'gpt-3.5-turbo-0613', 'prompt_type': 'openai_chat', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': '', 'PreInput': None, 'PreResponse': '', 'terminate_response': [], 'chat_sep': '\n', 'chat_turn_sep': '\n', 'humanstr': None, 'botstr': None, 'generates_leading_space': False, 'system_prompt': '', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 4046, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'gpt-3.5-turbo-16k-0613', 'prompt_type': 'openai_chat', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': '', 'PreInput': None, 'PreResponse': '', 'terminate_response': [], 'chat_sep': '\n', 'chat_turn_sep': '\n', 'humanstr': None, 'botstr': None, 'generates_leading_space': False, 'system_prompt': '', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 16335, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'gpt-35-turbo-1106', 'prompt_type': 'openai_chat', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': '', 'PreInput': None, 'PreResponse': '', 'terminate_response': [], 'chat_sep': '\n', 'chat_turn_sep': '\n', 'humanstr': None, 'botstr': None, 'generates_leading_space': False, 'system_prompt': '', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 16335, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'gpt-4-1106-preview', 'prompt_type': 'openai_chat', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': '', 'PreInput': None, 'PreResponse': '', 'terminate_response': [], 'chat_sep': '\n', 'chat_turn_sep': '\n', 'humanstr': None, 'botstr': None, 'generates_leading_space': False, 'system_prompt': '', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 127950, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'gemini-pro', 'prompt_type': 'google', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': '', 'PreInput': None, 'PreResponse': '', 'terminate_response': [], 'chat_sep': '\n', 'chat_turn_sep': '\n', 'humanstr': None, 'botstr': None, 'generates_leading_space': False, 'system_prompt': 'I am a helpful assistant.  I will accurately answer all your questions.', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 32718, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'mistral-small-latest', 'prompt_type': 'mistralai', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': '', 'PreInput': None, 'PreResponse': '', 'terminate_response': [], 'chat_sep': '\n', 'chat_turn_sep': '\n', 'humanstr': None, 'botstr': None, 'generates_leading_space': False, 'system_prompt': '', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 32718, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'mistral-large-latest', 'prompt_type': 'mistralai', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': '', 'PreInput': None, 'PreResponse': '', 'terminate_response': [], 'chat_sep': '\n', 'chat_turn_sep': '\n', 'humanstr': None, 'botstr': None, 'generates_leading_space': False, 'system_prompt': '', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 32718, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
{'base_model': 'mistral-medium', 'prompt_type': 'mistralai', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': '', 'PreInput': None, 'PreResponse': '', 'terminate_response': [], 'chat_sep': '\n', 'chat_turn_sep': '\n', 'humanstr': None, 'botstr': None, 'generates_leading_space': False, 'system_prompt': '', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None, 'llamacpp_dict': {'n_gpu_layers': 100, 'use_mlock': True, 'n_batch': 1024, 'n_gqa': 0, 'model_path_llama': '', 'model_name_gptj': '', 'model_name_gpt4all_llama': '', 'model_name_exllama_if_no_config': ''}, 'rope_scaling': {}, 'max_seq_len': 32718, 'max_output_seq_len': None, 'exllama_dict': {}, 'gptq_dict': {}, 'attention_sinks': False, 'sink_dict': {}, 'truncation_generation': False, 'hf_model_dict': {}}
"""

## Prompt 3: Language Translation

In [ ]:
## havent thought about it yet lol

TODO:  
  
Output email .txt  
input user name for email sign-off?  
Encapsulate all functions  
Language Translation prompt  

In [ ]:
# with open("trial_meeting_minutes_output.txt","w") as f:
#     f.write(llm_reply.content)

# Emailing stuff that I havent looked at yet

## Convert output file into pdf file to attach to email

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def create_pdf(text, filename='output.pdf'):
    # Create a canvas
    c = canvas.Canvas(filename, pagesize=letter)
    
    # Set up font
    c.setFont("Helvetica", 12)
    
    # Split text into lines
    lines = text.split('\n')
    
    # Set starting y-coordinate
    y = 750
    
    # Write text to the canvas
    for line in lines:
        c.drawString(50, y, line)
        y -= 15  # Adjust for next line
        
    # Save the canvas
    c.save()


In [ ]:
# Create PDF
create_pdf(llm_reply.content, "trial_meeting_minutes_output.pdf")

Automate Email sending

https://keentolearn.medium.com/automating-emails-with-python-a-comprehensive-guide-ba00fa98b92#:~:text=Once%20you%20have%20set%20up,account%2C%20and%20send%20the%20email.

## Created junk email for everyone to use
email : gpt10.4213.1@outlook.com <br>
pw: gpt10email<br>

name : gpt<br>
surname : 10<br>
country : senegal<br>
dob : 7 Mar 1989

#### Send email without attachment

In [ ]:
# #This works!
# import smtplib

# smtp_server = 'smtp-mail.outlook.com'
# smtp_port = 587
# smtp_username = 'gpt10.4213.1@outlook.com'
# smtp_password = 'gpt10email'

# from_email = 'gpt10.4213.1@outlook.com'
# to_email = 'gpt10.4213.1@outlook.com'
# subject = 'Hello, world!'
# body = 'This is a test email. using smtplib~~'

# message = f'Subject: {subject}\n\n{body}'

# with smtplib.SMTP(smtp_server, smtp_port) as smtp:
#     smtp.starttls()
#     smtp.login(smtp_username, smtp_password)
#     smtp.sendmail(from_email, to_email, message)

In [ ]:
smtp_server_mapping = {
    "gmail":"smtp.gmail.com",
    "outlook":"smtp-mail.outlook.com",
    "hotmail":"smtp.office365.com",
    "yahoo":"smtp.mail.yahoo.com",
}  #the rest dont care first since different smtp_port # ....

#### Send email with attachment

works with a list of email addresses ~ish

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

smtp_server = 'smtp-mail.outlook.com'
smtp_port = 587
smtp_username = 'gpt10.4213.1@outlook.com'
smtp_password = 'gpt10email'

from_email = 'gpt10.4213.1@outlook.com'
to_email = 'gpt10.4213.1@outlook.com'
to_email_list = ['gpt10.4213.1@outlook.com',]

meeting_date = "5/3/2024"
subject = f"{meeting_date} Meeting"  # meeting_date either user input or LLM search through document.
body = f'Please find attached the report. \n This is a summary of the meeting. \n {llm_reply2.content}'

msg = MIMEMultipart()
msg['From'] = from_email
# msg['To'] = to_email # send to single user
msg['To'] = ", ".join(to_email_list) # send to multiple users
msg['Subject'] = subject
msg.attach(MIMEText(body))

with open('trial_meeting_minutes_output.pdf', 'rb') as f:
    attachment = MIMEApplication(f.read(), _subtype='pdf')
    attachment.add_header('Content-Disposition', 'attachment', filename='report.pdf')
    msg.attach(attachment)

with open('trial_meeting_minutes_output.txt', 'rb') as f:
    attachment = MIMEApplication(f.read(), _subtype='txt')
    attachment.add_header('Content-Disposition', 'attachment', filename='report_friendly.txt')
    msg.attach(attachment)

with smtplib.SMTP(smtp_server, smtp_port) as smtp:
    smtp.starttls()
    smtp.login(smtp_username, smtp_password)
    smtp.send_message(msg)